# Notebook content

In [ ]:
batch_size = 32
plantdata = PlantDataModule(TRAIN_DATASET_PATH, TEST_DATASET_PATH, batch_size)

In [ ]:
trainer, model, test_result = train_model(
    model_name= "TLEfficientnet",
    datamodule= plantdata,
    max_epochs=10,
    model_hparams={"num_classes": 12},
    optimizer_name="Adam",
    optimizer_hparams={"lr": 1e-3, "weight_decay": 1e-4},
    metrics_name= ["Accuracy", "Precision", "Recall", "F1Score"],
    metrics_hparams= [
        {"task": "multiclass", "num_classes": 12, "average": "macro"},
        {"task": "multiclass", "num_classes": 12, "average": "macro"},
        {"task": "multiclass", "num_classes": 12, "average": "macro"},
        {"task": "multiclass", "num_classes": 12, "average": "macro"}
    ]
)

In [ ]:
# Load the TensorBoard extension
%load_ext tensorboard
%tensorboard --logdir /saved_models/TLEfficientnet/logs/

import IPython

display(IPython.display.HTML('''
<button id='open_tb'>Open TensorBoard</button>
<button id='hide_tb'>Hide TensorBoard</button>
<script>document.querySelector('#open_tb').onclick = () => { window.open(document.querySelector('iframe').src, "__blank") }
        document.querySelector('#hide_tb').onclick = () => { document.querySelector('iframe').style.display = "none" }</script>'''))

## Make prediction

In [ ]:
preds = trainer.predict(model, plantdata)

In [ ]:
classes = os.listdir(TRAIN_DATASET_PATH)
idx_to_class_dict = {i: _class for i, _class in enumerate(classes)}
species = np.vectorize(lambda idx: idx_to_class_dict[idx])(torch.cat(preds, 0))

## Create submission file

In [ ]:
import os 
import numpy as np
import pandas as pd
TEST_DATASET_PATH = os.environ.get("PATH_TEST_DATASETS", "/kaggle/input/plant-seedlings-classification/test")

In [ ]:
species = ["Charklock"] * len(os.listdir(TEST_DATASET_PATH))

In [ ]:
submission = pd.DataFrame({"file": os.listdir(TEST_DATASET_PATH),
                         "species": species})

submission.to_csv("submission.csv", index=False, header=True)